In [9]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
import random
import os
from IPython.display import display, HTML
import csv
from tqdm.notebook import tqdm

In [2]:
## 2023-2024 Coaches

afc_head_coaches = [
    {"team": "Baltimore Ravens", "coach": "John Harbaugh", "style": "Harbaugh's teams are known for their physicality and toughness. On offense, they've recently been known for a unique, run-heavy attack led by QB Lamar Jackson. On defense, they're traditionally aggressive, often blitzing to generate pressure."},
    {"team": "Buffalo Bills", "coach": "Sean McDermott", "style": "McDermott is a defensive-minded coach. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been balanced, with a recent emphasis on the passing game with QB Josh Allen."},
    {"team": "Cincinnati Bengals", "coach": "Zac Taylor", "style": "Taylor is an offensive-minded coach with a background in the Sean McVay coaching tree. His offenses typically feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Cleveland Browns", "coach": "Kevin Stefanski", "style": "Stefanski is known for his run-heavy offensive scheme that features a lot of two tight end sets and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Denver Broncos", "coach": "Sean Payton", "style": "Payton is another offensive-minded coach known for his aggressive play-calling. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Houston Texans", "coach": "DeMeco Ryans", "style": "Ryans, a former player, is a defensive-minded coach. His defenses are typically aggressive, often blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Indianapolis Colts", "coach": "Shane Steichen", "style": "Steichen is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Jacksonville Jaguars", "coach": "Doug Pederson", "style": "Pederson is an offensive-minded coach known for his aggressive play-calling. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Kansas City Chiefs", "coach": "Andy Reid", "style": "Reid is an offensive-minded coach known for his innovative and aggressive play-calling. His offenses often feature a lot of pre-snap motion and misdirection to create favorable matchups. On defense, his teams have typically played a lot of zone coverage."},
    {"team": "Las Vegas Raiders", "coach": "Josh McDaniels", "style": "McDaniels is an offensive-minded coach known for his innovative play-calling. His offenses often feature a strong passing attack with a lot of pre-snap motion and misdirection. On defense, his teams have typically been more conservative, often playing a lot of zone coverage."},
    {"team": "Los Angeles Chargers", "coach": "Brandon Staley", "style": "Staley is a defensive-minded coach known for his innovative schemes. His defenses often feature a lot of pre-snap movement and disguise. On offense, his teams have been more balanced, with a recent emphasis on the passing game with QB Justin Herbert."},
    {"team": "Miami Dolphins", "coach": "Mike McDaniel", "style": "McDaniel is an offensive-minded coach known for his work with the running game. His offenses often feature a lot of pre-snap motion and misdirection. On defense, his teams have typically been more conservative, often playing a lot of zone coverage."},
    {"team": "New England Patriots", "coach": "Bill Belichick", "style": "Belichick is known for his defensive mind and his ability to adapt his game plan based on the opponent. He's known for taking away an opponent's best weapon and forcing them to play left-handed. On offense, his teams have been known to use a balanced attack, with a strong emphasis on the short passing game."},
    {"team": "New York Jets", "coach": "Robert Saleh", "style": "Saleh is a defensive-minded coach known for his energy and motivational skills. His defenses are typically aggressive, often blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Pittsburgh Steelers", "coach": "Mike Tomlin", "style": "Tomlin's teams are known for their physicality and discipline. On offense, they typically feature a balanced attack with a strong emphasis on the running game. On defense, they are known for their aggressive, blitz-heavy schemes."},
    {"team": "Tennessee Titans", "coach": "Mike Vrabel", "style": "Vrabel's teams are known for their physicality and discipline. On offense, they typically feature a run-heavy attack. On defense, they are known for their aggressive, blitz-heavy schemes."}
]

nfc_head_coaches = [
    {"team": "Arizona Cardinals", "coach": "Jonathan Gannon", "style": "Gannon is a defensive-minded coach known for his adaptability and player development skills. His defenses often feature a lot of pre-snap movement and disguise. On offense, his teams have been more balanced."},
    {"team": "Atlanta Falcons", "coach": "Arthur Smith", "style": "Smith is known for his offensive strategies, particularly his run-heavy approach. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Carolina Panthers", "coach": "Frank Reich", "style": "Reich is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Chicago Bears", "coach": "Matt Eberflus", "style": "Eberflus is a defensive-minded coach known for his emphasis on effort and hustle. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Dallas Cowboys", "coach": "Mike McCarthy", "style": "McCarthy is an offensive-minded coach known for his work with quarterbacks. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Detroit Lions", "coach": "Dan Campbell", "style": "Campbell is known for his motivational skills and player-centric approach. His teams are typically physical and tough, with a focus on the running game on offense. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Green Bay Packers", "coach": "Matt LaFleur", "style": "LaFleur is an offensive-minded coach known for his innovative play-calling. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Los Angeles Rams", "coach": "Sean McVay", "style": "McVay is an offensive guru known for his innovative play-calling and use of pre-snap motion. His offenses often feature a strong passing attack with a lot of play-action. On defense, his teams have typically played a lot of zone coverage."},
    {"team": "Minnesota Vikings", "coach": "Kevin O’Connell", "style": "O'Connell is known for his work with quarterbacks and his offensive strategies. His offenses typically feature a balanced attack with a lot of play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "New Orleans Saints", "coach": "Dennis Allen", "style": "Allen is a defensive-minded coach known for his attention to detail and player development skills. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "New York Giants", "coach": "Brian Daboll", "style": "Daboll is known for his offensive strategies and his work with quarterbacks. His offenses often feature a strong passing attack with a lot of variety in formations and personnel groupings. On defense, his teams have typically been aggressive, often blitzing to generate pressure."},
    {"team": "Philadelphia Eagles", "coach": "Nick Sirianni", "style": "Sirianni is an offensive-minded coach known for his energy and player development skills. His offenses often feature a balanced attack with a lot of pre-snap motion and misdirection. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "San Francisco 49ers", "coach": "Kyle Shanahan", "style": "Shanahan is known for his offensive genius, particularly his run-heavy schemes. His offenses often feature a lot of pre-snap motion and play-action passes. On defense, his teams have been more conservative, often playing a lot of zone coverage."},
    {"team": "Seattle Seahawks", "coach": "Pete Carroll", "style": "Carroll's teams are known for their physical, run-first offense and aggressive, fast defense. His defensive philosophy is based on a 'bend but don't break' mentality, often giving up short gains but stiffening in the red zone."},
    {"team": "Tampa Bay Buccaneers", "coach": "Todd Bowles", "style": "Bowles is a defensive-minded coach known for his aggressive schemes. His defenses often feature a lot of blitzing to generate pressure. On offense, his teams have been more conservative, often focusing on the running game."},
    {"team": "Washington Commanders", "coach": "Ron Rivera", "style": "Rivera is known for his leadership and defensive strategies. His defenses are typically disciplined and fundamentally sound, often playing a lot of zone coverage. On offense, his teams have been more conservative, often focusing on the running game."}
]

print(len(afc_head_coaches))
print(len(nfc_head_coaches))

16
16


In [3]:
# Combine both AFC and NFC head coaches into one list
all_head_coaches = afc_head_coaches + nfc_head_coaches

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(all_head_coaches)

# Function to format the "style" column as multi-line text
def styled_display(row):
    return f"{row['style']}"

#     coach = f"<b>{row['coach']}</b>"
#     style = f"{row['style']}"
#     return f"{coach}<br>{style}"


# Apply the function to the "style" column to format it as multi-line text
df['style'] = df.apply(styled_display, axis=1)

# Set the option to display all rows and columns without truncation
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Display the DataFrame with HTML formatting
display(HTML(df.to_html(index=False)))

# Reset the option to the default values
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")


# Get Coach URL's

In [10]:
## Scrape head coach names URL's

url = 'https://www.pro-football-reference.com/coaches/'  # Replace with your URL

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the coaches
table = soup.find('table', {'id': 'coaches'})

# Initialize list to store coaches
coaches = []

# Loop through each row in the table
for row in table.find_all('tr'):
    # Initialize a dictionary to store coach data
    coach = {}

    # Extract the coach name and href link
    name_link = row.find('a')
    if name_link:
        coach['name'] = name_link.text
        coach['link'] = name_link['href']

    # Extract the number of super bowls
    super_bowls = row.find('td', {'data-stat': 'championships_super_bowl'})
    if super_bowls and super_bowls.text.strip():
        coach['super_bowls'] = int(super_bowls.text)

    # Extract the number of wins and losses
    wins = row.find('td', {'data-stat': 'wins'})
    if wins and wins.text.strip():
        coach['wins'] = int(wins.text)

    losses = row.find('td', {'data-stat': 'losses'})
    if losses and losses.text.strip():
        coach['losses'] = int(losses.text)

    # Add coach to list if data was found
    if coach:
        coaches.append(coach)
        
#print(f'There have been {coaches.count} head coaches in the NFL') 
print(f'There are {len(coaches)} coaches.')

for coach in coaches:
    print(coach)
        
# Create a DataFrame from the list of coaches
df = pd.DataFrame(coaches)

# Create new 'url' column
df['url'] = 'https://www.pro-football-reference.com' + df['link']

# Save the DataFrame to a CSV file
df.to_csv('./data-coaches/head_coaches_general.csv', index=False)


There are 518 coaches.
{'name': 'Don Shula', 'link': '/coaches/ShulDo0.htm', 'super_bowls': 2, 'wins': 328, 'losses': 156}
{'name': 'George Halas', 'link': '/coaches/HalaGe0.htm', 'super_bowls': 0, 'wins': 318, 'losses': 148}
{'name': 'Bill Belichick', 'link': '/coaches/BeliBi0.htm', 'super_bowls': 6, 'wins': 298, 'losses': 152}
{'name': 'Tom Landry', 'link': '/coaches/LandTo0.htm', 'super_bowls': 2, 'wins': 250, 'losses': 162}
{'name': 'Andy Reid', 'link': '/coaches/ReidAn0.htm', 'super_bowls': 2, 'wins': 247, 'losses': 138}
{'name': 'Curly Lambeau', 'link': '/coaches/LambCu0.htm', 'super_bowls': 0, 'wins': 226, 'losses': 132}
{'name': 'Paul Brown', 'link': '/coaches/BrowPa0.htm', 'super_bowls': 0, 'wins': 213, 'losses': 104}
{'name': 'Marty Schottenheimer', 'link': '/coaches/SchoMa0.htm', 'super_bowls': 0, 'wins': 200, 'losses': 126}
{'name': 'Chuck Noll', 'link': '/coaches/NollCh0.htm', 'super_bowls': 4, 'wins': 193, 'losses': 148}
{'name': 'Dan Reeves', 'link': '/coaches/ReevDa0.ht

# Loop Through Coaches

In [12]:
# Coaching Results
df = pd.read_csv('./data-coaches/head_coaches_general.csv')

# Get the total number of rows in the DataFrame
total_rows = len(df)

# Initialize the progress bar
for index, row in tqdm(df.iterrows(), total=total_rows):
    url = row['url']
    name = row['name']

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div containing the table
    table_wrapper = soup.find('div', id='all_coaching_results')

    # Find the table within the div (assuming it's the first table)
    table = table_wrapper.find('table')

    # Create a filename using the name
    filename = f"./data-coaches/coaching-results/{name.replace(' ', '_')}.csv"

    # Open a CSV file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        # Iterate through the rows of the table
        for row in table.find_all('tr'):
            # Get all the columns in the row (both th and td elements)
            columns = row.find_all(['th', 'td'])

            # Extract the text from each column
            row_data = [col.text for col in columns]

            # Write the row data to the CSV file
            writer.writerow(row_data)

    print(f'CSV file has been written successfully for {name}.')
    
    time.sleep(5)


  0%|          | 0/518 [00:00<?, ?it/s]

CSV file has been written successfully for Don Shula.
CSV file has been written successfully for George Halas.
CSV file has been written successfully for Bill Belichick.
CSV file has been written successfully for Tom Landry.
CSV file has been written successfully for Andy Reid.
CSV file has been written successfully for Curly Lambeau.
CSV file has been written successfully for Paul Brown.
CSV file has been written successfully for Marty Schottenheimer.
CSV file has been written successfully for Chuck Noll.
CSV file has been written successfully for Dan Reeves.
CSV file has been written successfully for Chuck Knox.
CSV file has been written successfully for Jeff Fisher.
CSV file has been written successfully for Bill Parcells.
CSV file has been written successfully for Tom Coughlin.
CSV file has been written successfully for Mike Shanahan.
CSV file has been written successfully for Mike Tomlin.
CSV file has been written successfully for Pete Carroll.
CSV file has been written successful

CSV file has been written successfully for John Ralston.
CSV file has been written successfully for Anthony Lynn.
CSV file has been written successfully for Eric Mangini.
CSV file has been written successfully for Dick Rauch.
CSV file has been written successfully for Romeo Crennel.
CSV file has been written successfully for Adam Gase.
CSV file has been written successfully for Pop Ivy.
CSV file has been written successfully for Tony Sparano.
CSV file has been written successfully for Mike Tice.
CSV file has been written successfully for Paddy Driscoll.
CSV file has been written successfully for Jim Mora.
CSV file has been written successfully for Neill Armstrong.
CSV file has been written successfully for Dutch Clark.
CSV file has been written successfully for Walt Kiesling.
CSV file has been written successfully for John Mackovic.
CSV file has been written successfully for Mike McCormack.
CSV file has been written successfully for Jim Schwartz.
CSV file has been written successfully 

CSV file has been written successfully for Rod Marinelli.
CSV file has been written successfully for Mike Pettine.
CSV file has been written successfully for Kay Stephenson.
CSV file has been written successfully for Brian Daboll.
CSV file has been written successfully for Jack Faulkner.
CSV file has been written successfully for Al Groh.
CSV file has been written successfully for John Mazur.
CSV file has been written successfully for Mike McDaniel.
CSV file has been written successfully for Cap McEwan.
CSV file has been written successfully for Nelson Talbot.
CSV file has been written successfully for Joe Vitt.
CSV file has been written successfully for Charlie Waller.
CSV file has been written successfully for Bob Waterfield.
CSV file has been written successfully for Bud Wilkinson.
CSV file has been written successfully for Steve Wilks.
CSV file has been written successfully for Doc Alexander.
CSV file has been written successfully for Ed Biles.
CSV file has been written successfull

CSV file has been written successfully for Chuck Studley.
CSV file has been written successfully for Gus Tebell.
CSV file has been written successfully for Joe Thomas.
CSV file has been written successfully for Mel Tucker.
CSV file has been written successfully for Rick Venturi.
CSV file has been written successfully for Carl Voyles.
CSV file has been written successfully for Willie Wilkin.
CSV file has been written successfully for Ray Willsey.
CSV file has been written successfully for Larry Wilson.
CSV file has been written successfully for Hoge Workman.
CSV file has been written successfully for Shorty Barr.
CSV file has been written successfully for Tom Bettis.
CSV file has been written successfully for Hugo Bezdek.
CSV file has been written successfully for Fred Bruney.
CSV file has been written successfully for Cam Cameron.
CSV file has been written successfully for Bill Conkright.
CSV file has been written successfully for Mel Doherty.
CSV file has been written successfully for

In [8]:
## Team Ranks

# Read the CSV file with pandas
df = pd.read_csv('./data-coaches/head_coaches_general.csv')

# Loop through the rows in the DataFrame
for index, row in df.iterrows():
    url = row['url']
    name = row['name']

    # Send a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the div containing the table
    table_wrapper = soup.find('div', id='all_coaching_ranks')

    # Find the table within the div (assuming it's the first table)
    table = table_wrapper.find('table')

    # Create a filename using the name
    filename = f"./data-coaches/team-ranks/{name.replace(' ', '_')}_coaching_ranks.csv"

    # Open a CSV file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        # Iterate through the rows of the table
        for row in table.find_all('tr'):
            # Get all the columns in the row (both th and td elements)
            columns = row.find_all(['th', 'td'])

            # Extract the text from each column
            row_data = [col.text for col in columns]

            # Write the row data to the CSV file
            writer.writerow(row_data)

    print(f'CSV file has been written successfully for {name}.')
    
    time.sleep(5)
    

AttributeError: 'NoneType' object has no attribute 'find_all'